In [1]:
#必要なライブラリをインポート
import numpy as np
import pandas as pd
import re


class Timer:
    """
    Timer measures elapsed time taken to execute a code block.
    It is recommended to use this object with `with` block.
    
    Usage:
        with Timer(message):
            # code block to be measured here...
    """
    def __init__(self, msg: str):
        self.msg = msg
        
    def __enter__(self):
        print(self.msg, end='')
        self.start = time.time()
        return self

    def __exit__(self, *args):
        elapsed = time.time() - self.start
        print(' done: took %.2f sec.' % elapsed)

In [ ]:
#東京都23区の物件を個別に保存しておいたものを読み込み、結合
csv_files = ['suumo_central_east.csv', 'suumo_west_north_south.csv']
data = [pd.read_csv(file, encoding='utf-8') for file in csv_files]
df = pd.concat(data, axis=0, ignore_index=True).reset_index(drop=True)

print('Shape of df:', df.shape)

#立地を「最寄駅」と「徒歩〜分」に分割
splitted1 = df['立地1'].str.split(' 歩', expand=True)
splitted1.columns = ['立地11', '立地12']
splitted2 = df['立地2'].str.split(' 歩', expand=True)
splitted2.columns = ['立地21', '立地22']
splitted3 = df['立地3'].str.split(' 歩', expand=True)
splitted3.columns = ['立地31', '立地32']

#その他費用を、「敷金」「礼金」「保証金」「敷引,償却」に分割
splitted4 = df['敷/礼/保証/敷引,償却'].str.split('/', expand=True)
splitted4.columns = ['敷金', '礼金', '保証金', '敷引償却']

#分割したカラムを結合
df = pd.concat([df, splitted1, splitted2, splitted3, splitted4], axis=1)

#分割前のカラムは分析に使用しないので削除しておく
df.drop(['立地1','立地2','立地3','敷/礼/保証/敷引,償却'], axis=1, inplace=True)
#変な立地情報を削除する
df.loc[df['立地31'] == '自01/02自由が丘駅-駒大/深沢小学校', ['立地31']] = None

#数値として扱いたいので、不要な文字列を削除
df['築年数'] = df['築年数'].str.replace('新築', '0') #新築は築年数0年とする
df['築年数'] = df['築年数'].str.replace(r'築(\d+)年', r'\1')
# df['築年数'] = df['築年数'].str.replace('年', '')
df['賃料'] = df['賃料'].str.replace(u'万円', u'')
df['管理費'] = df['管理費'].str.replace(u'円', u'')
df['専有面積'] = df['専有面積'].str.replace('m2', '')
df['立地12'] = df['立地12'].str.replace(u'分', u'')
df['立地22'] = df['立地22'].str.replace(u'分', u'')
df['立地32'] = df['立地32'].str.replace(u'分', u'')
df['敷金'] = df['敷金'].str.replace(u'万円', u'')
df['礼金'] = df['礼金'].str.replace(u'万円', u'')
df['保証金'] = df['保証金'].str.replace(u'万円', u'')
df['敷引償却'] = df['敷引償却'].str.replace(u'万円', u'')
df['敷引償却'] = df['敷引償却'].replace('実費', 0) #「実費」と文字列が入っている場合がある

df.head()

In [19]:
#「-」を0に変換
df['管理費'] = df['管理費'].replace('-', 0)
df['敷金'] = df['敷金'].replace('-', 0)
df['礼金'] = df['礼金'].replace('-', 0)
df['保証金'] = df['保証金'].replace('-', 0)
df['敷引償却'] = df['敷引償却'].replace('-', 0)

#文字列から数値に変換
df['賃料'] = pd.to_numeric(df['賃料'])
df['管理費'] = pd.to_numeric(df['管理費'])
df['敷金'] = pd.to_numeric(df['敷金'])
df['礼金'] = pd.to_numeric(df['礼金'])
df['保証金'] = pd.to_numeric(df['保証金'])
df['敷引償却'] = pd.to_numeric(df['敷引償却'])
df['築年数'] = pd.to_numeric(df['築年数'])
df['専有面積'] = pd.to_numeric(df['専有面積'])
df['立地12'] = pd.to_numeric(df['立地12'])
df['立地22'] = pd.to_numeric(df['立地22'])
df['立地32'] = pd.to_numeric(df['立地32'])

#単位を合わせるために、管理費以外を10000倍。
df['賃料'] = df['賃料'] * 10000
df['敷金'] = df['敷金'] * 10000
df['礼金'] = df['礼金'] * 10000
df['保証金'] = df['保証金'] * 10000
df['敷引償却'] = df['敷引償却'] * 10000

#管理費は実質的には賃料と同じく毎月支払うことになるため、「賃料+管理費」を家賃を見る指標とする
df['賃料+管理費'] = df['賃料'] + df['管理費']
#敷金/礼金と保証金は同じく初期費用であり、どちらかが適用されるため、合計を初期費用を見る指標とする
df['敷/礼/保証金'] = df['敷金'] + df['礼金'] + df['保証金']

df.head()

,マンション名,住所,築年数,建物高さ,階,賃料,管理費,間取り,専有面積,立地11,...,立地21,立地22,立地31,立地32,敷金,礼金,保証金,敷引償却,賃料+管理費,敷/礼/保証金
0,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13階建,7階,159000.0,15000,3LDK,58.17,ＪＲ総武線/錦糸町駅,...,都営新宿線/住吉駅,11.0,ＪＲ総武線/亀戸駅,14.0,159000.0,159000.0,0.0,0.0,174000.0,318000.0
1,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13階建,6階,168000.0,15000,3LDK,62.92,ＪＲ総武線/錦糸町駅,...,都営新宿線/住吉駅,11.0,ＪＲ総武線/亀戸駅,14.0,168000.0,168000.0,0.0,0.0,183000.0,336000.0
2,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13階建,3階,181000.0,15000,3LDK,70.52,ＪＲ総武線/錦糸町駅,...,都営新宿線/住吉駅,11.0,ＪＲ総武線/亀戸駅,14.0,181000.0,181000.0,0.0,0.0,196000.0,362000.0
3,銀座レジデンス伍番館,東京都中央区湊２,15,13階建,2階,120000.0,0,1DK,31.62,ＪＲ京葉線/八丁堀駅,...,東京メトロ日比谷線/八丁堀駅,7.0,東京メトロ有楽町線/新富町駅,6.0,120000.0,0.0,0.0,0.0,120000.0,120000.0
4,銀座レジデンス伍番館,東京都中央区湊２,15,13階建,4階,128000.0,0,1DK,32.24,ＪＲ京葉線/八丁堀駅,...,東京メトロ日比谷線/八丁堀駅,7.0,東京メトロ有楽町線/新富町駅,6.0,128000.0,0.0,0.0,0.0,128000.0,128000.0


In [22]:
#住所を「東京都」「〜区」「市町村番地」に分割
splitted6 = df['住所'].str.split('区', expand=True)
splitted6.columns = ['区', '市町村']
splitted6['区'] = splitted6['区'] + '区'
splitted6['区'] = splitted6['区'].str.replace('東京都', '')

#立地を「路線」「駅」「徒歩〜分」に分割
splitted7 = df['立地11'].str.split('/', expand=True)
splitted7.columns = ['路線1', '駅1']
splitted7['徒歩1'] = df['立地12']
splitted8 = df['立地21'].str.split('/', expand=True)
splitted8.columns = ['路線2', '駅2']
splitted8['徒歩2'] = df['立地22']
splitted9 = df['立地31'].str.split('/', expand=True)
splitted9.columns = ['路線3', '駅3']
splitted9['徒歩3'] = df['立地32']

#結合
df = pd.concat([df, splitted6, splitted7, splitted8, splitted9], axis=1)
#不要なカラムを削除
df.drop(['立地11','立地12','立地21','立地22','立地31','立地32'], axis=1, inplace=True)

#階を数値化。地下はマイナスとして扱う
splitted10 = df['階'].str.split('-', expand=True)
splitted10.columns = ['階1', '階2']
splitted10['階1'] = splitted10['階1'].str.replace(u'階', u'')
splitted10['階1'] = splitted10['階1'].str.replace(u'B', u'-')
splitted10['階1'] = pd.to_numeric(splitted10['階1'])

df = pd.concat([df, splitted10], axis=1)

#建物高さを数値化。地下は無視。
df['建物高さ'] = pd.to_numeric(df['建物高さ'].str.replace(r'(地下\d+地上)?(\d+)階建', r'\2').str.replace(u'平屋', u'1'))

#indexを振り直す（これをしないと、以下の処理でエラーが出る）
df = df.reset_index(drop=True)
df.head()

,マンション名,住所,築年数,建物高さ,階,賃料,管理費,間取り,専有面積,敷金,...,駅1,徒歩1,路線2,駅2,徒歩2,路線3,駅3,徒歩3,階1,階2
0,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13,7階,159000.0,15000,3LDK,58.17,159000.0,...,錦糸町駅,9.0,都営新宿線,住吉駅,11.0,ＪＲ総武線,亀戸駅,14.0,7.0,None
1,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13,6階,168000.0,15000,3LDK,62.92,168000.0,...,錦糸町駅,9.0,都営新宿線,住吉駅,11.0,ＪＲ総武線,亀戸駅,14.0,6.0,None
2,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13,3階,181000.0,15000,3LDK,70.52,181000.0,...,錦糸町駅,9.0,都営新宿線,住吉駅,11.0,ＪＲ総武線,亀戸駅,14.0,3.0,None
3,銀座レジデンス伍番館,東京都中央区湊２,15,13,2階,120000.0,0,1DK,31.62,120000.0,...,八丁堀駅,7.0,東京メトロ日比谷線,八丁堀駅,7.0,東京メトロ有楽町線,新富町駅,6.0,2.0,None
4,銀座レジデンス伍番館,東京都中央区湊２,15,13,4階,128000.0,0,1DK,32.24,128000.0,...,八丁堀駅,7.0,東京メトロ日比谷線,八丁堀駅,7.0,東京メトロ有楽町線,新富町駅,6.0,4.0,None


In [23]:
#間取りを「部屋数」「DK有無」「K有無」「L有無」「S有無」に分割
df['間取りDK'] = 0
df['間取りK'] = 0
df['間取りL'] = 0
df['間取りS'] = 0
df['間取り'] = df['間取り'].str.replace(u'ワンルーム', u'1') #ワンルームを1に変換

for symb in ['DK', 'K', 'L', 'S']:
    df.loc[df['間取り'].str.contains(symb), '間取り' + symb] = 1

df['間取り'] = df['間取り'].str.replace(r'[DKLS]', '')
df['間取り'] = pd.to_numeric(df['間取り'])
df

,マンション名,住所,築年数,建物高さ,階,賃料,管理費,間取り,専有面積,敷金,...,徒歩2,路線3,駅3,徒歩3,階1,階2,間取りDK,間取りK,間取りL,間取りS
0,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13,7階,159000.0,15000,3,58.17,159000.0,...,11.0,ＪＲ総武線,亀戸駅,14.0,7.0,None,1,1,1,0
1,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13,6階,168000.0,15000,3,62.92,168000.0,...,11.0,ＪＲ総武線,亀戸駅,14.0,6.0,None,1,1,1,0
2,エコロジー錦糸町レジデンス,東京都墨田区江東橋４,13,13,3階,181000.0,15000,3,70.52,181000.0,...,11.0,ＪＲ総武線,亀戸駅,14.0,3.0,None,1,1,1,0
3,銀座レジデンス伍番館,東京都中央区湊２,15,13,2階,120000.0,0,1,31.62,120000.0,...,7.0,東京メトロ有楽町線,新富町駅,6.0,2.0,None,1,1,0,0
4,銀座レジデンス伍番館,東京都中央区湊２,15,13,4階,128000.0,0,1,32.24,128000.0,...,7.0,東京メトロ有楽町線,新富町駅,6.0,4.0,None,1,1,0,0
5,銀座レジデンス伍番館,東京都中央区湊２,15,13,12階,136000.0,0,1,33.91,136000.0,...,7.0,東京メトロ有楽町線,新富町駅,6.0,12.0,None,1,1,0,0
6,銀座レジデンス伍番館,東京都中央区湊２,15,13,7階,179000.0,0,2,50.38,179000.0,...,7.0,東京メトロ有楽町線,新富町駅,6.0,7.0,None,1,1,1,0
7,パークアクシス辰巳ステージ,東京都江東区辰巳２,11,13,2階,96000.0,8000,1,28.48,96000.0,...,15.0,りんかい線,東雲駅,19.0,2.0,None,0,1,0,0
8,パークアクシス辰巳ステージ,東京都江東区辰巳２,11,13,2階,96000.0,8000,1,28.48,96000.0,...,15.0,りんかい線,東雲駅,19.0,2.0,None,0,1,0,0
9,パークアクシス辰巳ステージ,東京都江東区辰巳２,11,13,1階,113000.0,8000,1,30.62,113000.0,...,15.0,りんかい線,東雲駅,19.0,1.0,None,0,1,0,0


In [25]:
#カラムを入れ替えて、csvファイルとして出力
df = df[['マンション名','住所','区','市町村','間取り','間取りDK','間取りK','間取りL','間取りS','築年数','建物高さ','階1',
         '専有面積','敷/礼/保証金','路線1','駅1','徒歩1','路線2','駅2','徒歩2','路線3','駅3','徒歩3',
         '賃料','管理費','賃料+管理費','敷金','礼金','保証金','敷引償却']]

df.to_csv('suumo_tokyo.csv', encoding='utf-8')